In [4]:
import os
import json

from myo_input_tk import *
from unity_sim_tk import *
from controller_input_tk import *

from adpframe_tk import *
from adp_tk_tab12 import *
from adp_tk_tab3 import *

from adp_tk_init_train4 import *
from adp_tk_new_train import *
from adp_tk_deployment4 import *

In [5]:
class App(tk.Tk):
    def __init__(self, parent=None):
        tk.Tk.__init__(self, parent)
        self.parent = parent
        self.initialize()
 
    def initialize(self):
        self.title("System")
        self.geometry('800x720+10+10')
        self.resizable(width=True, height=True) #设置窗口是否可变长、宽

        self.default_set_path = 'default_settings.json'
        self.default_para_path = 'default_paras.json'
        self.sets = {
            "folder": "E:/Masterthesis/codes/storage/", 
            "name": "name01", 
            "myo_sdk": "E:/Masterthesis/codes/myo/myo-sdk-win-0.9.0/", 
            "unity_port": "9000"
        }
        self.paras = {
            # feature extraction
            'window_size': 40,
            # training
            'modify_scale': 1,
            'output_scale': 3,
            # clustering
            'cluster_range_scale': (20,10),
            'c_min_len': 3,
            'time_interval': 5,
            'w1': 0.1,
            'w2': 0.05,
            'p_threshold': 0.4,
            'eps': 3e-7,
            'num_train': 1
        }

        self.unity_joint = None
        self.adp1 = AdpFramework()
        # self.reach = False
        self.id_mode = 0

        self.test = []


        # Create menu bar
        menu_bar = tk.Menu(self)
        self.config(menu=menu_bar)

        # Create menu file
        menuFile = tk.Menu(menu_bar, tearoff=False)
        menu_bar.add_cascade(label=" File ", menu=menuFile)

        # Tab
        # Tab style
        style = ttk.Style(self)
        style.configure(style='custom.TNotebook', tabposition='nw')

        # Create tab
        tab_main = ttk.Notebook(self, style='custom.TNotebook')
        tab_main.place(relx=0.02, rely=0.02, relwidth=0.96, relheight=0.95)

        tab1=tk.Frame(tab_main)
        tab_main.add(tab1, text=' Settings ')

        tab2=tk.Frame(tab_main)
        tab_main.add(tab2, text=' Parameters ')

        tab3=tk.Frame(tab_main)
        tab_main.add(tab3, text=' Workspace ')

        tab_main3 = ttk.Notebook(tab3)
        tab_main3.grid(row=20, column=0, padx=5, pady=5, sticky='w')

        tab3_1=tk.Frame(tab_main3)
        tab_main3.add(tab3_1, text=' EMG Feature ')

        tab3_2=tk.Frame(tab_main3)
        tab_main3.add(tab3_2, text=' IMU ')

        tab3_3=tk.Frame(tab_main3)
        tab_main3.add(tab3_3, text=' EMG ')

        self.tab3_4=tk.Frame(tab_main3)
        tab_main3.add(self.tab3_4, text=' Graph ')


        # Submenu
        menuFileImport = tk.Menu(menuFile, tearoff=False)
        menuFile.add_cascade(label='Import settings', menu=menuFileImport)
        menuFileImport.add_command(label='Import default', command=self.import_default_set)
        menuFileImport.add_command(label='Select File', command=self.select_file_set)

        menuFileSave = tk.Menu(menuFile, tearoff=False)
        menuFile.add_cascade(label='Save settings', menu=menuFileSave)
        menuFileSave.add_command(label='Save as default', command=self.save_as_default_set)
        menuFileSave.add_command(label='Save as', command=self.save_as_set)

        menuFile.add_separator()

        menuFileImport2 = tk.Menu(menuFile, tearoff=False)
        menuFile.add_cascade(label='Import parameters', menu=menuFileImport2)
        menuFileImport2.add_command(label='Import default', command=self.import_default_para)
        menuFileImport2.add_command(label='Select File', command=self.select_file_para)

        menuFileSave2 = tk.Menu(menuFile, tearoff=False)
        menuFile.add_cascade(label='Save parameters', menu=menuFileSave2)
        menuFileSave2.add_command(label='Save as default', command=self.save_as_default_para)
        menuFileSave2.add_command(label='Save as', command=self.save_as_para)

        
        b0_s = tk.Button(self, text='Switch Window Size', bg="white",command=self.switch_window_size, width=16, height=1)
        b0_s.grid(row=0, column=0, padx=360, pady=0, sticky='w')
        
        
        # Tab1 - frame0
        t0_0 = tk.Label(tab1, text="Storage", font=("Calibri", 12, "bold"))
        t0_0.grid(row=0, column=0, padx=15, pady=(15,0), sticky='w')

        frame0=tk.Frame(tab1, bg="#DCDCDC", width=460, height=180)
        frame0.grid(row=1, column=0, padx=10, pady=5, sticky='wn', rowspan=4, columnspan=2)
        frame0.grid_propagate(0)

        t0_1 = tk.Label(frame0, bg="#DCDCDC", text="Folder path:")
        t0_1.grid(row=1, column=0, padx=5, pady=5, sticky='w')

        self.e0_1 = tk.Entry(frame0, width=50)
        self.e0_1.insert(0, self.sets['folder'])
        self.e0_1.grid(row=1, column=1, padx=5, pady=5, sticky='w', columnspan=3)

        t0_2 = tk.Label(frame0, bg="#DCDCDC", text="User name:")
        t0_2.grid(row=2, column=0, padx=5, pady=5, sticky='w')

        self.e0_2 = tk.Entry(frame0, width=20)
        self.e0_2.insert(0, self.sets['name'])
        self.e0_2.grid(row=2, column=1, padx=5, pady=5, sticky='w')

        b0_1 = tk.Button(frame0, text='Confirm', bg="white",command=self.confirm_storage, width=10, height=1)
        b0_1.grid(row=3, column=0, padx=5, pady=0, sticky='w')

        self.txt0_1 = tk.StringVar()
        self.txt0_1.set("Empty path. Please click 'Confirm'.\n \n ")
        self.t0_3 = tk.Label(frame0, textvariable=self.txt0_1, bg="#DCDCDC", font=("Calibri Light", 11), justify='left')
        self.t0_3.grid(row=4, column=0, padx=5, pady=5, sticky='w', rowspan=2, columnspan=3)

        self.txt0_2 = tk.StringVar()
        self.t0_4 = tk.Label(frame0, textvariable=self.txt0_2, bg="#DCDCDC", font=("Calibri Light", 11), justify='left')
        self.t0_4.grid(row=6, column=0, padx=5, pady=5, sticky='w', columnspan=3)

        
        # Tab1 - frame1
        t1_0 = tk.Label(tab1, text="Connection", font=("Calibri", 12, "bold"))
        t1_0.grid(row=10, column=0, padx=15, pady=(5,0), sticky='w')

        frame1=tk.Frame(tab1, bg="#DCDCDC", width=460, height=135)
        frame1.grid(row=11, column=0, padx=10, pady=5, sticky='wn', rowspan=3, columnspan=2)
        frame1.grid_propagate(0)

        t1_1 = tk.Label(frame1, bg="#DCDCDC", text="Myo Armband")
        t1_1.grid(row=0, column=0, padx=5, pady=5, sticky='w', columnspan=2)

        t1_2 = tk.Label(frame1, bg="#DCDCDC", text="SDK path (Please enter path to ../myo-sdk-win-0.9.0/):")
        t1_2.grid(row=1, column=0, padx=5, pady=0, sticky='w')

        self.e1_1 = tk.Entry(frame1, width=50)
        self.e1_1.insert(0, self.sets['myo_sdk'])
        self.e1_1.grid(row=2, column=0, padx=5, pady=0, sticky='w')

        b1_1 = tk.Button(frame1, text='Connect', bg="white", command=self.connect_myo, width=10, height=1)
        b1_1.grid(row=2, column=1, padx=5, pady=0, sticky='w')

        self.txt1_1 = tk.StringVar()
        self.txt1_1.set("Click 'Connect' to connect Myo.\n ")
        self.t1_3 = tk.Label(frame1, bg="#DCDCDC", textvariable=self.txt1_1, font=("Calibri Light", 11), justify='left')
        self.t1_3.grid(row=3, column=0, padx=5, pady=5, sticky='w', rowspan=2, columnspan=2)

        # Tab1 - frame2
        frame2 = tk.Frame(tab1, bg="#DCDCDC", width=460, height=115)
        frame2.grid(row=20, column=0, padx=10, pady=5, sticky='w', rowspan=2, columnspan=3)
        frame2.grid_propagate(0)

        t2_1 = tk.Label(frame2, bg="#DCDCDC", text="Unity")
        t2_1.grid(row=0, column=0, padx=5, pady=5, sticky='w', columnspan=2)

        t2_2 = tk.Label(frame2, bg="#DCDCDC", text="Listen prot:")
        t2_2.grid(row=1, column=0, padx=5, pady=5, sticky='w')

        self.e2_0 = tk.Entry(frame2, width=10)
        self.e2_0.insert(0, self.sets['unity_port'])
        self.e2_0.grid(row=1, column=1, padx=5, pady=5, sticky='w')

        b2_1 = tk.Button(frame2, text='Connect', bg="white",command=self.connect_unity, width=10, height=1)
        b2_1.grid(row=1, column=2, padx=5, pady=0, sticky='w')

        self.txt2_1 = tk.StringVar()
        self.txt2_1.set("Press 'Connect' to connect Unity.\n ")
        self.t2_3 = tk.Label(frame2, bg="#DCDCDC",textvariable=self.txt2_1, font=("Calibri Light", 11), justify='left')
        self.t2_3.grid(row=2, column=0, padx=5, pady=5, sticky='w', columnspan=3)


        # Tab1 - frame3
        frame3 = tk.Frame(tab1, bg="#DCDCDC", width=460, height=110)
        frame3.grid(row=30, column=0, padx=10, pady=5, sticky='w', rowspan=2, columnspan=3)
        frame3.grid_propagate(0) 

        t3_1 = tk.Label(frame3, bg="#DCDCDC", text="VR Controller")
        t3_1.grid(row=0, column=0, padx=5, pady=5, sticky='w')

        b3_1 = tk.Button(frame3, text='Connect', bg="white",command=self.check_vr, width=10, height=1)
        b3_1.grid(row=1, column=0, padx=6, pady=0, sticky='w')

        self.txt3_1 = tk.StringVar()
        self.txt3_1.set("Press 'Connect' to connect VR controller.\n ")
        self.t3_3 = tk.Label(frame3, bg="#DCDCDC",textvariable=self.txt3_1, font=("Calibri Light", 11), justify='left')
        self.t3_3.grid(row=2, column=0, padx=5, pady=5, sticky='w', columnspan=2)


        # Tab2 Parameters
        # Tab2 - frame2_1 - Feature Extraction
        frame2_1 = tk.Frame(tab2, bg="#DCDCDC", width=600, height=80)
        frame2_1.grid(row=0, column=0, padx=10, pady=5, sticky='w', rowspan=2, columnspan=3)
        frame2_1.grid_propagate(0) 

        t2_1 = tk.Label(frame2_1, bg="#DCDCDC", text="Feature Extraction")
        t2_1.grid(row=0, column=0, padx=5, pady=5, sticky='w')

        # window_size
        t2_2 = tk.Label(frame2_1, bg="#DCDCDC", text="window_size: ")
        t2_2.grid(row=1, column=0, padx=5, pady=5, sticky='e')

        self.e2_1 = tk.Entry(frame2_1, width=10)
        self.e2_1.insert(0, self.paras['window_size'])
        self.e2_1.grid(row=1, column=1, padx=5, pady=5, sticky='w', columnspan=3)


        # Tab2 - frame2_4 - Scales
        frame2_4 = tk.Frame(tab2, bg="#DCDCDC", width=600, height=120)
        frame2_4.grid(row=2, column=0, padx=10, pady=5, sticky='w', rowspan=2, columnspan=3)
        frame2_4.grid_propagate(0) 

        t2_9 = tk.Label(frame2_4, bg="#DCDCDC", text="Scales")
        t2_9.grid(row=0, column=0, padx=5, pady=5, sticky='w')

        # scale
        t2_9 = tk.Label(frame2_4, bg="#DCDCDC", text="modify_synergy_scale: ")
        t2_9.grid(row=1, column=0, padx=5, pady=5, sticky='e')

        self.e2_9 = tk.Entry(frame2_4, width=10)
        self.e2_9.insert(0, self.paras['modify_scale'])
        self.e2_9.grid(row=1, column=1, padx=5, pady=5, sticky='w', columnspan=3)

        t2_10 = tk.Label(frame2_4, bg="#DCDCDC", text="output_motion_scale: ")
        t2_10.grid(row=2, column=0, padx=5, pady=5, sticky='e')

        self.e2_10 = tk.Entry(frame2_4, width=10)
        self.e2_10.insert(0, self.paras['output_scale'])
        self.e2_10.grid(row=2, column=1, padx=5, pady=5, sticky='w', columnspan=3)


        # Tab2 - frame2_2 - Clustering
        frame2_2 = tk.Frame(tab2, bg="#DCDCDC", width=600, height=200)
        frame2_2.grid(row=10, column=0, padx=10, pady=5, sticky='w', rowspan=2, columnspan=3)
        frame2_2.grid_propagate(0) 

        t2_1 = tk.Label(frame2_2, bg="#DCDCDC", text="Clustering")
        t2_1.grid(row=0, column=0, padx=5, pady=5, sticky='w')

        # cluster_range_scale
        t2_2 = tk.Label(frame2_2, bg="#DCDCDC", text="cluster_range_scale: ")
        t2_2.grid(row=1, column=0, padx=5, pady=(5,0), sticky='e')

        self.e2_2_1 = tk.Entry(frame2_2, width=4)
        self.e2_2_1.insert(0, self.paras['cluster_range_scale'][0])
        self.e2_2_1.grid(row=1, column=1, padx=5, pady=(5,0), sticky='w', columnspan=3)

        t2_2_2 = tk.Label(frame2_2, bg="#DCDCDC", text="-")
        t2_2_2.grid(row=1, column=1, padx=35, pady=(5,0), sticky='w')

        self.e2_2_2 = tk.Entry(frame2_2, width=4)
        self.e2_2_2.insert(0, self.paras['cluster_range_scale'][1])
        self.e2_2_2.grid(row=1, column=1, padx=45, pady=(5,0), sticky='w', columnspan=3)

        # c_min_len
        t2_3 = tk.Label(frame2_2, bg="#DCDCDC", text="cluster_min_len: ")
        t2_3.grid(row=2, column=0, padx=5, pady=0, sticky='e')

        self.e2_3 = tk.Entry(frame2_2, width=10)
        self.e2_3.insert(0, self.paras['c_min_len'])
        self.e2_3.grid(row=2, column=1, padx=5, pady=0, sticky='w', columnspan=3)

        # w1
        t2_4 = tk.Label(frame2_2, bg="#DCDCDC", text="w1: ")
        t2_4.grid(row=3, column=0, padx=5, pady=0, sticky='e')

        self.e2_4 = tk.Entry(frame2_2, width=10)
        self.e2_4.insert(0, self.paras['w1'])
        self.e2_4.grid(row=3, column=1, padx=5, pady=0, sticky='w', columnspan=3)

        t2_4_2 = tk.Label(frame2_2, bg="#DCDCDC", text=" (Weight of predictions based on transition): ")
        t2_4_2.grid(row=3, column=2, padx=5, pady=0, sticky='w')

        # w2
        t2_5 = tk.Label(frame2_2, bg="#DCDCDC", text="w2: ")
        t2_5.grid(row=4, column=0, padx=5, pady=0, sticky='e')

        self.e2_5 = tk.Entry(frame2_2, width=10)
        self.e2_5.insert(0, self.paras['w2'])
        self.e2_5.grid(row=4, column=1, padx=5, pady=0, sticky='w', columnspan=3)

        t2_5_2 = tk.Label(frame2_2, bg="#DCDCDC", text=" (Weight of cluster modifications in new training)")
        t2_5_2.grid(row=4, column=2, padx=5, pady=0, sticky='w')


        # p_threshold
        t2_6 = tk.Label(frame2_2, bg="#DCDCDC", text="p_threshold: ")
        t2_6.grid(row=5, column=0, padx=5, pady=0, sticky='e')

        self.e2_6 = tk.Entry(frame2_2, width=10)
        self.e2_6.insert(0, self.paras['p_threshold'])
        self.e2_6.grid(row=5, column=1, padx=5, pady=0, sticky='w', columnspan=3)

        t2_6_2 = tk.Label(frame2_2, bg="#DCDCDC", text=" (Prob below the threshold is considered as a new cluster.)")
        t2_6_2.grid(row=5, column=2, padx=5, pady=0, sticky='w')


        # eps
        t2_7 = tk.Label(frame2_2, bg="#DCDCDC", text="eps: ")
        t2_7.grid(row=6, column=0, padx=5, pady=0, sticky='e')

        self.e2_7 = tk.Entry(frame2_2, width=10)
        self.e2_7.insert(0, self.paras['eps'])
        self.e2_7.grid(row=6, column=1, padx=5, pady=0, sticky='w', columnspan=3)


        # Tab2 - frame2_3 - Data Acquisition
        frame2_3 = tk.Frame(tab2, bg="#DCDCDC", width=600, height=100)
        frame2_3.grid(row=30, column=0, padx=10, pady=5, sticky='w', rowspan=2, columnspan=3)
        frame2_3.grid_propagate(0) 

        t2_8_2 = tk.Label(frame2_3, bg="#DCDCDC", text="Data Acquisition")
        t2_8_2.grid(row=0, column=0, padx=5, pady=5, sticky='w')

        # sample_time_interval
        self.t2_8 = tk.Label(frame2_3, bg="#DCDCDC", text="sample_time_interval (ms): ")
        self.t2_8.grid(row=1, column=0, padx=5, pady=5, sticky='e')

        self.e2_8 = tk.Entry(frame2_3, width=10)
        self.e2_8.insert(0, self.paras['time_interval'])
        self.e2_8.grid(row=1, column=1, padx=5, pady=5, sticky='w', columnspan=3)

        # training data size
        self.t2_11 = tk.Label(frame2_3, bg="#DCDCDC", text="initial training repeat: ")
        self.t2_11.grid(row=2, column=0, padx=5, pady=5, sticky='e')

        self.e2_11 = tk.Entry(frame2_3, width=10)
        self.e2_11.insert(0, self.paras['num_train'])
        self.e2_11.grid(row=2, column=1, padx=5, pady=5, sticky='w', columnspan=3)


        # Tab2 - button
        b2_1 = tk.Button(tab2, text='Confirm', bg="white",command=self.confirm_para, width=10, height=1)
        b2_1.grid(row=40, column=0, padx=15, pady=5, sticky='w')

        self.txt2_9 = tk.StringVar()
        self.txt2_9.set("Press 'Confirm' to confirm all the parameters.")
        self.t2_9 = tk.Label(tab2, textvariable=self.txt2_9, font=("Calibri Light", 11), justify='left')
        self.t2_9.grid(row=41, column=0, padx=15, pady=5, sticky='w', columnspan=2)


        # Tab3
        b4_1 = tk.Button(tab3, text='Initial \nTraining', bg="white",command=self.initial_train, width=8, height=2)
        b4_1.grid(row=0, column=0, padx=5, pady=0, sticky='w')

        b4_2 = tk.Button(tab3, text='New \nTraining', bg="white",command=self.new_train, width=8, height=2)
        b4_2.grid(row=0, column=0, padx=75, pady=0, sticky='w')

        b4_3 = tk.Button(tab3, text='Testing', bg="white",command=self.deployment, width=8, height=2)
        b4_3.grid(row=0, column=0, padx=145, pady=0, sticky='w')

        b4_3 = tk.Button(tab3, text='Confirm \nfor Unity', bg="#bcbcbc",command=self.start_unity, width=8, height=2)
        b4_3.grid(row=0, column=0, padx=215, pady=0, sticky='w')

        self.txt4_1 = tk.StringVar()
        self.txt4_1.set("Check the 'device connection' and 'storage path' before you start.")
        self.t4_3 = tk.Label(tab3, textvariable=self.txt4_1, bg="#DCDCDC", font=("Calibri Light", 11), justify='left')
        self.t4_3.grid(row=1, column=0, padx=5, pady=5, sticky='w', columnspan=3)

        t4_4_1 = tk.Label(tab3, text="Store to File:")
        t4_4_1.grid(row=0, column=0, padx=320, pady=5, sticky='wn')

        self.e4_1 = tk.Entry(tab3, width=4)
        self.e4_1.insert(0, '01')
        self.e4_1.grid(row=0, column=0, padx=320, pady=(30,0), sticky='wn')

        b4_4 = tk.Button(tab3, text='Copy', bg="white",command=self.confirm_copy, width=4, height=1)
        b4_4.grid(row=0, column=0, padx=360, pady=(25,0), sticky='wn')
        
        b4_5 = tk.Button(tab3, text='Load', bg="white",command=self.confirm_load, width=4, height=1)
        b4_5.grid(row=0, column=0, padx=400, pady=(25,0), sticky='wn')


        # # reacdhe   这块需要加
        # t4_4_1 = tk.Label(tab3, text="Target point was ")
        # t4_4_1.grid(row=10, column=0, padx=5, pady=0, sticky='w')

        # self.txt4_4 = tk.StringVar()
        # self.txt4_4.set("not decided")
        # self.t4_4_2 = tk.Label(tab3, textvariable=self.txt4_4, font=("Calibri Light", 11), justify='left')
        # # self.t4_4_2.config(fg="orange")
        # self.t4_4_2.grid(row=10, column=0, padx=110, pady=5, sticky='w')

        # t4_4_3 = tk.Label(tab3, text=" in the last training.")
        # t4_4_3.grid(row=10, column=0, padx=190, pady=0, sticky='w')


        # checkbox
        self.show = {}
        cb = {}
        for i in range(8):
            self.show[i] = tk.BooleanVar()
            cb[i] = tk.Checkbutton(tab3_3, text="CH" + str(i+1), variable=self.show[i])
            cb[i].grid(row=0, column=1, padx=5, pady=5+i*20, sticky='wn')

        self.show[0].set(True)

        # plot emg
        fig0, self.line0, ax0 = self.show_graph(np.zeros((8,40)), (6,4), [0,40], [-100,100])
        self.graph0 = FigureCanvasTkAgg(fig0, master=tab3_3)
        canvas0 = self.graph0.get_tk_widget()
        canvas0.grid(row=0, column=0)
        self.graph0.draw()


        # plot imu
        fig1, self.line1, ax1 = self.show_graph(np.zeros((4,40)), (6,4), [0,40], [-2,2])
        self.graph1 = FigureCanvasTkAgg(fig1, master=tab3_2)
        canvas1 = self.graph1.get_tk_widget()
        canvas1.grid(row=0, column=0)
        self.graph1.draw()


        # checkbox
        self.show2_1 = {}
        cb2_1 = {}
        for i in range(8):
            self.show2_1[i] = tk.BooleanVar()
            cb2_1[i] = tk.Checkbutton(tab3_1, text="CH" + str(i+1), variable=self.show2_1[i])
            cb2_1[i].grid(row=0, column=1, padx=5, pady=5+i*20, sticky='wn')

        self.show2_1[0].set(True)

        self.ax_ylim = tk.DoubleVar()
        slider = tk.Scale(tab3_1, from_=1, to=0.02, resolution=0.02, variable=self.ax_ylim, command=self.zoom_in)
        slider.set(1)
        slider.grid(row=0, column=1, padx=5, pady=240)


        frame_ft = tk.Frame(tab3_1, bg="#DCDCDC", width=80, height=240)
        frame_ft.grid(row=0, column=2, padx=5, pady=5, sticky='wn')
        frame_ft.grid_propagate(0) 

        # checkbox
        cb2_2 = {}
        # cb2_2, cb2_3 = {}, {}
        ft_name = ['VAR', 'RMS', 'LOGDET', 'WL', 'DASDV', 'ZC', 'MNP', 'TOT', 'MNF', 'MDF']
        # ft1_name = ['VAR', 'RMS', 'LOG_DET', 'WL', 'DASDV', 'ZC']
        # ft2_name = ['MNP', 'TOT', 'MNF', 'MDF']

        self.var_cb = tk.IntVar()
        self.var_cb.set(0) # 设置默认选项为1

        for i in range(10):
            cb2_2[i] = tk.Radiobutton(frame_ft, text=ft_name[i], variable=self.var_cb, value=i, bg="#DCDCDC")
            cb2_2[i].grid(row=0, column=2, padx=5, pady=5+i*20, sticky='wn')
        # for i in range(4):
        #     cb2_3[i] = tk.Radiobutton(frame_ft, text=ft2_name[i], variable=self.var_cb, value=i+6, bg="#DCDCDC")
        #     cb2_3[i].grid(row=0, column=3, padx=5, pady=5+i*20, sticky='wn')


        # plot emg_ft
        fig2, self.line2, self.ax2 = self.show_graph(np.zeros((8,40)), (6,4), [0,40], [-1.2,1.2])
        self.graph2 = FigureCanvasTkAgg(fig2, master=tab3_1)
        canvas2 = self.graph2.get_tk_widget()
        canvas2.grid(row=0, column=0, sticky='wn')
        self.graph2.draw()


    # Func - Tab1
    def import_sets(self):
        import_sets(self)


    def import_paras(self):
        import_paras(self)


    def get_settings(self):
        return get_settings(self)


    def import_default_set(self):
        import_default_set(self)

    def select_file_set(self):
        select_file_set(self)


    def save_as_default_set(self):
        save_as_default_set(self)


    def save_as_set(self):
        save_as_set(self)
    
    
    def get_paras(self):
        return get_paras(self)


    def import_default_para(self):
        import_default_para(self)

    def select_file_para(self):
        select_file_para(self)


    def save_as_default_para(self):
        save_as_default_para(self)


    def save_as_para(self):
        save_as_para(self)


    # Func - tab2 - frame0
    def confirm_storage(self):
        confirm_storage(self)

    # Tab2 - frame1-3
    def connect_myo(self):
        connect_myo(self)

    def connect_unity(self):
        self.print_tk(self.txt2_1, self.t2_3, "Connecting Unity...\n ")
        port = self.e2_0.get()
        self.unity_joint, txt_print, connect = connect_unity_tk(port=port)
        self.txt2_1.set(txt_print)
        if connect:
            self.t2_3.config(fg="green")
        else:
            self.t2_3.config(fg="red")

    def check_vr(self):
        check_vr(self)

    def confirm_para(self):
        confirm_para(self)

    
    # Tab3 - graph
    def show_graph(self, data, figsize, xlim, ylim):
        return show_graph(self, data, figsize, xlim, ylim)


    def show_graph_emg(self, data, line, show):
        show_graph_emg(self, data, line, show)


    def show_graph_imu(self, data_que, line):
        show_graph_imu(self, data_que, line)


    def show_graph_emg_ft(self, data, line, show_1, var_cb):
        show_graph_emg_ft(self, data, line, show_1, var_cb)
    

    # adpframe paras
    def adp_paras(self):
        adp_paras(self)
    
    
    #tab3
    def initial_train(self):
        initial_train(self)
        

    def initial_train_start(self, hub, listener):
        initial_train_start(self, hub, listener)

            
    def initial_train_offline(self, data_df, m_label, txt_print):
        initial_train_offline(self, data_df, m_label, txt_print)

    
    def new_train(self):
        new_train(self)


    def new_train_start(self, cluster_read, adja_mat, emg_ft_mean, synergy_mean, txt_print):
        new_train_start(self, cluster_read, adja_mat, emg_ft_mean, synergy_mean, txt_print)
        

    def new_train_offline(self, txt_print, data_df2):
        new_train_offline(self, txt_print, data_df2)

    
    def deployment(self):
        deployment(self)
        
    def confirm_copy(self):
        confirm_copy(self)

    def confirm_load(self):
        confirm_load(self)

    def start_unity(self):
        if self.unity_joint == None:
            self.print_tk(self.txt4_1, self.t4_3, "Please first connect Unity", color="red")
        else:
            self.print_tk(self.txt4_1, self.t4_3, "Unity input is connected.", color="green")
            self.unity_joint.SetText(text1="Please select the mode.", text2=" ", color="black")
            self.id_mode = 2
            while True:
                controller = self.unity_joint.ReturnInput()
                if controller == 6:
                    self.unity_joint.SetText(text1="You exit unity connection", text2="Start again in Python.", color="red")
                    self.id_mode = 0
                    self.unity_joint.SetColor(id=self.id_mode)
                    break
                if controller == 3:
                    self.id_mode += 10
                    self.unity_joint.SetColor(id=self.id_mode)
                    if self.id_mode == 12:
                        self.initial_train()
                    elif self.id_mode == 13:
                        self.new_train()
                    elif self.id_mode == 14:
                        self.deployment()
                        
                if self.id_mode in [2,3,4]:
                    self.id_mode = switch_mode(controller, self.id_mode)
                    self.unity_joint.SetColor(id=self.id_mode)

                time.sleep(0.05)
        self.print_tk(self.txt4_1, self.t4_3, "End of Unity input.\nPress 'Confirm for Unity' to start again.", color="black")

    def zoom_in(self, value):
        ylim = self.ax_ylim.get()
        self.ax2.set_ylim([-ylim, ylim])
        self.graph2.draw()


    def print_tk(self, txt_var, t_label, txt_print, color=None):
        txt_var.set(txt_print)
        if color != None:
            t_label.config(fg=color)
        t_label.update()
    
    def print_unity(self, text1=" ", text2=" ", color="black", id_init=None):
        if self.unity_joint != None:
            self.unity_joint.SetText(text1=text1, text2=text2, color=color)
            if id_init != None:
                self.id_mode = id_init
                self.unity_joint.SetColor(id=self.id_mode)

    def switch_window_size(self):
        if self.winfo_width() + self.winfo_height()>=1200:
            self.geometry('500x200')
        else:
            self.geometry('800x720')
    
    def keyboard_func(self, event):
        # print("event.char =", event.char)
        # print("event.keycode =", event.keycode)
        if event.char=='u':
            self.initial_train()
        elif event.char=='i':
            self.new_train()
        elif event.char=='o':
            self.deployment()

        
        # if event.char=='9':
        #     # reached the target
        #     self.reach = True
        #     self.print_tk(self.txt4_4, self.t4_4_2, "REACHED", color="green")
            
        # elif event.char=='0':
        #     self.reach = False
        #     self.print_tk(self.txt4_4, self.t4_4_2, "NOT reached", color="orange")


        # elif event.char=='8':
        #     print(self.reach)

In [6]:
if __name__ == "__main__":
    app = App()
    app.bind("<Key>", app.keyboard_func)
    app.mainloop()